In [2]:
import sys
import json

In [25]:
# Parse the map from a given filename
def parse_map(filename):
        with open(filename, "r") as f:
                return [[char for char in line] for line in f.read().rstrip("\n").split("\n")]

In [26]:
# Check if a row,col index pair is on the map
def valid_index(pos, n, m):
        return 0 <= pos[0] < n  and 0 <= pos[1] < m

In [5]:
# Find the possible moves from position (row, col)
def moves(map, row, col, visits):
        moves=((row+1,col), (row-1,col), (row,col-1), (row,col+1))

	# Return only moves that are within the board and legal (i.e. go through open space ".")
        return [ move for move in moves if valid_index(move, len(map), len(map[0])) 
                                        and (map[move[0]][move[1]] in ".@" ) 
                                        and (move not in visits)]

In [6]:
def euclid_dist(current_state, target_state):
    return ((current_state[0] - target_state[0])**2 + (current_state[1] - target_state[1])**2)**0.5

In [7]:
def priority_pop(fringe):
    min = fringe[0][3]
    index = 0 
    for i in range(len(fringe)):
        if fringe[i][3] < min:
            min = fringe[i][3]
            index = i
    return fringe.pop(index)


In [8]:
def get_direction(last_move, curr_move):
    if last_move[0] - curr_move[0] == 1:
        return "N"
    elif last_move[0] - curr_move[0] == -1:
        return "S"
    elif last_move[1] - curr_move[1] == 1:
        return "W"
    elif last_move[1] - curr_move[1] == -1:
        return "E"

In [9]:
# Perform search on the map
def search1(house_map):
        # Find pichu start position
        pichu_loc=[(row_i,col_i) for col_i in range(len(house_map[0])) for row_i in range(len(house_map)) if house_map[row_i][col_i]=="p"][0]
        # Find target position (need it to compute heuristic)
        target_loc=[(row_i,col_i) for col_i in range(len(house_map[0])) for row_i in range(len(house_map)) if house_map[row_i][col_i]=="@"][0]
        # Find measure of herutstic to target
        heuristic = euclid_dist(pichu_loc, target_loc)
        
        fringe=[(pichu_loc,0,'',heuristic)]
        visited = []

        while fringe:
            (curr_move, curr_dist, curr_dir, curr_cost)=priority_pop(fringe)
            for move in moves(house_map, *curr_move, visited):
                if house_map[move[0]][move[1]]=="@":
                    return curr_dist+1, curr_dir + get_direction(curr_move, move)
                else:
                    fringe.append((move, curr_dist + 1, curr_dir + get_direction(curr_move, move), euclid_dist(move, target_loc)+curr_dist + 1))
                visited.append(move)
        return 'Inf'

In [10]:
house_map = parse_map('map2.txt')

In [11]:
house_map

[['.', '.', '.', '.', 'X', 'X', 'X'],
 ['.', 'X', 'X', 'X', '.', '.', '.'],
 ['.', '.', '.', '.', 'X', '.', '.'],
 ['.', 'X', '.', 'X', '.', '.', '.'],
 ['.', 'X', '.', 'X', '.', 'X', '.'],
 ['p', 'X', '.', 'X', '@', 'X', '.']]

In [12]:
search1(house_map)

'Inf'

## Part 2: Hide and Seek

In [66]:
# Parse the map from a given filename
def parse_map(filename):
	with open(filename, "r") as f:
		return [[char for char in line] for line in f.read().rstrip("\n").split("\n")]

In [67]:
# Count total # of pichus on board
def count_pichus(board):
    return sum([ row.count('p') for row in board ] )

In [68]:
# Return a string with the board rendered in a human-pichuly format
def printable_board(board):
    return "\n".join([ "".join(row) for row in board])

In [69]:
# Add a pichu to the board at the given position, and return a new board (doesn't change original)
def add_pichu(board, row, col):
    return board[0:row] + [board[row][0:col] + ['p',] + board[row][col+1:]] + board[row+1:]

In [70]:
# Get list of successors of given board state
def successors(board):
    return [add_pichu(board, r, c) for r in range(0, len(board)) for c in range(0,len(board[0])) if (board[r][c] == '.')
                                                                                                 and(valid_spot(board, r, c))]

In [71]:
# Get list of successors of given board state
def super_successors(board):
    return [add_pichu(board, r, c) for r in range(0, len(board)) for c in range(0,len(board[0])) if (board[r][c] == '.')
                                                                                                 and(super_valid_spot(board, r, c))]

In [72]:
#check if you can see a pichu from a given spot
def valid_spot(board, row, col):
    #check left
    c = col
    while c >= 0:
        if board[row][c]=='.':
            c += -1
        elif board[row][c]=='X' or board[row][c]=='@':
            c = -1
        else:
            return False
    #check right
    c = col
    while c < len(board[row]):
        if board[row][c]=='.': 
            c += 1
        elif board[row][c]=='X' or board[row][c]=='@':
            c = len(board[row])
        else:
            return False
    #check up
    r = row
    while r >= 0:
        if board[r][col]=='.': 
            r += -1
        elif board[r][col]=='X' or board[r][col]=='@':
            r = -1
        else:
            return False
    #check down
    r = row
    while r < len(board):
        if board[r][col]=='.': 
            r += 1
        elif board[r][col]=='X' or board[r][col]=='@':
            r = len(board)
        else:
            return False
    return True



In [73]:
#check if you can see a pichu from a given spot
def super_valid_spot(board, row, col):
    max_c = len(board[row])
    max_r = len(board)
    #check left
    c = col
    while c >= 0:
        if board[row][c]=='.':
            c += -1
        elif board[row][c]=='X' or board[row][c]=='@':
            c = -1
        else:
            return False
    #check right
    c = col
    while c < max_c:
        if board[row][c]=='.': 
            c += 1
        elif board[row][c]=='X' or board[row][c]=='@':
            c = len(board[row])
        else:
            return False
    #check up
    r = row
    while r >= 0:
        if board[r][col]=='.': 
            r += -1
        elif board[r][col]=='X' or board[r][col]=='@':
            r = -1
        else:
            return False
    #check down
    r = row
    while r < max_r:
        if board[r][col]=='.': 
            r += 1
        elif board[r][col]=='X' or board[r][col]=='@':
            r = len(board)
        else:
            return False
    #check up and to the left
    c = col
    r = row
    while c >= 0 and r >= 0:
        if board[r][c]=='.':
            c += -1
            r += -1
        elif board[r][c]=='X' or board[r][c]=='@':
            c = -1
        else:
            return False
    #check up and to the right
    c = col
    r = row
    while c < max_c and r >= 0:
        if board[r][c]=='.':
            c += 1
            r += -1
        elif board[r][c]=='X' or board[r][c]=='@':
            r = -1
        else:
            return False
    #check down and to the left
    c = col
    r = row
    while c >= 0 and r < max_r:
        if board[r][c]=='.':
            c += -1
            r += 1
        elif board[r][c]=='X' or board[r][c]=='@':
            c = -1
        else:
            return False
    #check down and to the right
    c = col
    r = row
    while c < max_c and r < max_r:
        if board[r][c]=='.':
            c += 1
            r += 1
        elif board[r][c]=='X' or board[r][c]=='@':
            r = len(board)
        else:
            return False
    return True

In [74]:
# check if board is a goal state
def is_goal(board):
    return count_pichus(board) == K 

In [75]:
def is_super_goal(board, i):
    return count_pichus(board) == i

In [76]:
def standard_search(initial_board):
    fringe = [initial_board]
    while len(fringe) > 0:
        for s in successors(fringe.pop()):
            if is_goal(s):
                return(s)
            fringe.append(s)
    return None

In [77]:
def super_search(initial_board, i):
    fringe = [initial_board]
    while len(fringe) > 0:
        for s in super_successors(fringe.pop()):
            if is_super_goal(s, i):
                return(s)
            fringe.append(s)
    return None

In [78]:
# Solve!
def solve(initial_board):
    i = K
    if i > 0:
        return standard_search(initial_board)
    elif i == 0:
        i = 2
        while super_search(initial_board, i):
            i += 1
        i -= 1
        return super_search(initial_board, i)
    else:
        return "K must be greater than or equal to 0"

In [56]:
start_map = parse_map('map.txt')

In [33]:
super_map = add_pichu(start_map, 2, 1)
super_map

[['.', '.', '.', '.', 'X', 'X', 'X'],
 ['.', 'X', 'X', 'X', '.', '.', '.'],
 ['.', 'p', '.', '.', 'X', '.', '.'],
 ['.', 'X', '.', 'X', '.', '.', '.'],
 ['.', 'X', '.', 'X', '.', 'X', '.'],
 ['p', 'X', '.', 'X', '@', 'X', '.']]

In [89]:
K = 0
%time
solve(start_map)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


[['.', 'p', '.', '.', 'X', 'X', 'X'],
 ['.', 'X', 'X', 'X', '.', '.', '.'],
 ['.', '.', '.', 'p', 'X', 'p', '.'],
 ['.', 'X', '.', 'X', '.', '.', '.'],
 ['.', 'X', 'p', 'X', '.', 'X', 'p'],
 ['p', 'X', '.', '.', 'p', 'X', '@']]